## data_clean_sales

### clean weekly sales data

#### challenges in raw data:
1. entries are recorded on different days of the week
2. some weeks have multiple entires
3. between start date and end date, about 20% of all weeks are missing

    
#### solutions to get clean data:
1. normalize dates: since we are dealing with weekly sales, convert all dates to the Monday of that week
2. consolidate all records from a week into one dated the Monday of that week by adding their weekly sales together.
3. for each missing week, create a record dated the Monday of that week; impute its weekly sales value as follows:
    * if it's a non-holiday week, use the weekly sales value from the closeset available non-holiday week
    * else (it's a holiday week), use the mean weekly sales from the same holiday if exist,otherwise use the closest avaiable holiday weekly sales. 
    
#### clean data and save in pickles

In [34]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pylab as plt
%matplotlib inline

import statsmodels.api as sm 

In [35]:
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler

In [36]:
def week_of_year(date):
    return date.week

In [49]:
import datetime

# since we are dealing with weekly data, on which day an entry got recorded is not important
# rather, we will normalize them by converting them to the date of Monday of the same week.
# later we will aggregate weekly sales by summing all the entry on the same normalized date.
def year_week(date):
    temp = date.isocalendar()
    date_string = str(temp[0])+' '+str(temp[1])+' 1'
    return datetime.strptime(date_string, '%G %V %u')

In [40]:
# read data 
sales=pd.read_csv('data/sales.csv')


# normalize dates
sales['Date'] = pd.to_datetime(sales['Date'])
sales['Date']=sales['Date'].apply(year_week)

In [41]:
start_date=min(sales['Date'])
end_date=max(sales['Date'])
dates = sorted(list(pd.date_range(start_date, end_date, freq='W')))
all_dates=[year_week(start_date).date()]+[year_week(d).date() for d in dates]

In [42]:
# all the weeks when we have data
all_dates=list(set(all_dates))

In [43]:
all_depts=sorted(sales['Dept'].unique())

In [44]:
all_stores=sorted(sales['Store'].unique())

In [45]:
n_depts = len(all_depts)
n_stores = len(all_stores)

In [46]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-04-26,24924.50,False
1,1,1,2010-11-29,46039.49,True
2,1,1,2010-02-15,41595.55,False
3,1,1,2010-02-22,19403.54,False
4,1,1,2010-05-03,21827.90,False


In [47]:
import collections


# slice data; look at each (dept, store) combo
slice_df = sales.copy(deep=True)
groups = dict(list( slice_df.groupby(['Dept','Store'])))


holidays

In [50]:


from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime

cal = get_calendar('USFederalHolidayCalendar')  # Create calendar instance
holidays = list(cal.holidays(start_date, end_date))
#convert from actual holiday dates to the dates of first day of holiday weeks
holidays = [year_week(day) for day in holidays]


Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13 Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13 Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13 Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [51]:
major_holidays = {}

super_bowl_dates=[datetime(2010, 2, 12, 0, 0),datetime(2011, 2, 11, 0, 0),datetime(2012, 2, 10, 0, 0),datetime(2013, 2, 8, 0, 0)]
labor_day_dates=[datetime(2010, 9, 10, 0, 0),datetime(2011, 9, 9, 0, 0),datetime(2012, 9, 7, 0, 0),datetime(2013, 9, 6, 0, 0)]
thanksgiving_dates=[datetime(2010, 11, 26, 0, 0),datetime(2011, 11, 25, 0, 0),datetime(2012, 11, 23, 0, 0),datetime(2013, 11, 29, 0, 0)]
xmas_dates=[datetime(2010, 12, 25, 0, 0),datetime(2011, 12, 24, 0, 0),datetime(2012, 12, 22, 0, 0),datetime(2013, 12, 21, 0, 0)]

super_bowl_weeks = [year_week(d) for d in super_bowl_dates ]
labor_day_weeks= [year_week(d) for d in labor_day_dates ]
thanksgiving_weeks = [year_week(d) for d in thanksgiving_dates ]
xmas_weeks = [year_week(d) for d in xmas_dates ]

for week in super_bowl_weeks:
    major_holidays[week] = 'super_bowl'
for week in labor_day_weeks:
    major_holidays[week] = 'labor_day'
for week in thanksgiving_weeks:
    major_holidays[week] = 'thanksgiving'
for week in xmas_weeks:
    major_holidays[week] = 'xmas'



In [52]:
# converts dates to same form so they can be compared

def timestamp_to_date(d):
    if len(str(d)) < 13:
        return d
    date_string = str(d)
    date_string = str(d).split(' ')[0]
    return datetime.strptime(date_string,'%Y-%m-%d')

* segment data 
* for each (dept, store) combo, build a data frame then save this data frame in a dict with key (dept, store)
* note some (dept, store) do not exist

* each data frame holds actual info as well as imputed data for the missing weeks during this 2 year period.

* before imputation, weekly sales is set to 0 for missing weeks.

In [53]:
def is_holiday(date):
    return year_week(date) in holidays



dfs ={}
indices = all_dates
invalid_dept_store = []

for dept in range(1,n_depts+1):
    for store in range(1,n_stores+1):
        try:
            df = groups[(dept,store)]
            df = df[['Date', 'Weekly_Sales', 'IsHoliday']]
            
            existing_dates = [d.date() for d in df['Date']]
            missing_dates = [d for d in all_dates if d not in existing_dates]
            num_missing = len(missing_dates)
        
        
            df2=pd.DataFrame()
            df2['Date']=missing_dates
            df2['Weekly_Sales']=[0]*num_missing
        
            df2['IsHoliday']=df2['Date'].apply(is_holiday)
            df = pd.concat([df,df2], axis=0)
            df['Date']=df['Date'].apply(timestamp_to_date)
            df=df.set_index('Date')
            df=df.sort_index()
            # sum multiple entries in the same week
            df=df.groupby(level=0).sum()

            dfs[(dept,store)] = df
        except:
            # this key combo does not exist in data
            invalid_dept_store.append((dept,store))
        

In [54]:
major_holiday_means = {}

for dept,store in dfs.keys():
    df = dfs[(dept,store)]
    df_indices = df.index
    holiday_indices = major_holidays.keys()
    both = set(df_indices).intersection(set(holiday_indices))
    
    
    super_bowl_mean = np.mean([df['Weekly_Sales'][idx] for idx in both if major_holidays[idx]=='super_bowl' and df['Weekly_Sales'][idx]>0])
    labor_day_mean = np.mean([df['Weekly_Sales'][idx] for idx in both if major_holidays[idx]=='labor_day' and df['Weekly_Sales'][idx]>0])
    thanksgiving_mean = np.mean([df['Weekly_Sales'][idx] for idx in both if major_holidays[idx]=='thanksgiving' and df['Weekly_Sales'][idx]>0])
    xmas_mean =  np.mean([df['Weekly_Sales'][idx] for idx in both if major_holidays[idx]=='xmas' and df['Weekly_Sales'][idx]>0])
    
    major_holiday_means[(dept,store,'super_bowl')] = super_bowl_mean
    major_holiday_means[(dept,store,'labor_day')] = labor_day_mean
    major_holiday_means[(dept,store,'thanksgiving')] = thanksgiving_mean
    major_holiday_means[(dept,store,'xmas')] = xmas_mean
    


/home/ubuntu/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


clean data from given week in a dept at given store

In [55]:
def impute_one_weekly_sales(dept,store,date):
    try:
        df = dfs[(dept,store)]    
        indices = sorted(df.index)

        normal_weeks = [idx for idx in indices if df['IsHoliday'][idx]==False and df['Weekly_Sales'][idx]>0] 

        # date of value used for imputation for given date
        pos = min(range(len(normal_weeks)), key=lambda i: abs(normal_weeks[i]-date))
        imputation_date = normal_weeks[pos]

        imputation_value = df['Weekly_Sales'][imputation_date]
        # pass imputation by reference
        df['Weekly_Sales'][date] = imputation_value
                
        if date not in indices:
            print('invalid date')
            return
        
        if date not in major_holidays.keys(): #implement holiday cases later
        #if df[date]['IsHoliday']==False:
            
            normal_weeks = [idx for idx in indices if df['IsHoliday'][idx]==False and df['Weekly_Sales'][idx]>0] 
            
             # date of value used for imputation for given date
            imputation_date = min(enumerate(normal_weeks), key=lambda x: abs((normal_weeks[1]-date).days))[1]
            imputation_value = df['Weekly_Sales'][imputation_date]
            # pass imputation by reference
            df['Weekly_Sales'][date] = imputation_value
            return
        
        else:
            holiday = major_holidays[date]
            
            #print((dept,store,holiday))
            
            imputation_value = major_holiday_means[(dept,store,holiday)]
            #print('here')
            
            if pd.isnull(imputation_value):
                holiday_weeks = [idx for idx in indices if df['IsHoliday'][idx]==True and df['Weekly_Sales'][idx]>0]
                imputation_value = np.mean(df['Weekly_Sales'][holiday_weeks])
            df['Weekly_Sales'][date] = imputation_value
            return   
            
    except:
        print('invalid (dept,store)')

clean data from multiple weeks in a dept at a given store

In [56]:
def impute_multiple_weekly_sales(dept,store,dates):
    for d in dates:
        impute_one_weekly_sales(dept,store,d) 
    return

clean all data from a dept at a given store

In [57]:
def impute_all_weekly_sales(dept,store):
    
    print('impute: store =', store)
    
    df = dfs[(dept,store)]    
    indices = sorted(df.index)
        
    dates = [idx for idx in indices if df['Weekly_Sales'][idx]<0.01]
    impute_multiple_weekly_sales(dept,store,dates) 
    
    return

In [58]:
keys = dfs.keys()
stores_has_dept = {}
for dept in all_depts:
    stores_has_dept[dept] = []
for dept,store in keys:
    stores_has_dept[dept].append(store)
    

Clean data from given dept and save it in a pickle file

In [63]:
def clean_dept_sales(dept):
    
    store_list = stores_has_dept[dept]
    for store in store_list:
        impute_all_weekly_sales(dept, store)
     
    cleaned_sales_list= []
    for store in store_list:
        print('dept = ',dept, 'add to list: store = ',store)
        
        df = dfs[(dept,store)]
        df = df.reset_index()
        n_rows = df.shape[0]
        df['Store']= [store]*n_rows
        df['Dept']=[dept]*n_rows
        #rearrange order of columns
        df = df[list(sales.columns)]
        cleaned_sales_list.append(df)
    
    if (len(cleaned_sales_list)>0):
        result = pd.concat(cleaned_sales_list)
        result.to_pickle('data/clean_sales_dept_'+str(dept)+'.pkl')
    return 

Clean all sales data

In [64]:
for dept in all_depts:
    clean_dept_sales(dept)
